In [15]:
import rasterio
import numpy as np
import geopandas as gpd

from sklearn.preprocessing import MinMaxScaler, RobustScaler

In [25]:
# Specify paths
rasPath = "D:\\TEMP\\_output\_tif\\_Manning_Invasives\\Manning_Invasives_Structural_NEW.tif"
rasPathOut = "D:\\TEMP\\_output\_tif\\_Manning_Invasives\\Manning_Invasives_Structural_NEW_Scaled2.tif"

In [26]:
### MULTI

# Read raster. In this case it is a 10-band raster and we are only reading 5 bands
with rasterio.open(rasPath, 'r') as src:
    b, g, r, re, nir = src.read(2), src.read(4), src.read(6), src.read(8), src.read(10)
    array = np.stack([b,g,r,re,nir])
    array = array.astype(np.float32) # Read array
    array[array == src.nodata] = np.nan # Assign np.nan value to all cells that contain a nodata value
    
    affine = src.transform # Affine transform
    profile = src.profile # Metadata
    nodata = src.nodata # Nodata

In [17]:
### RGB

# Read raster. 
with rasterio.open(rasPath, 'r') as src:
    r, g, b = src.read(1), src.read(2), src.read(3)
    array = np.stack([r,g,b])
    array = array.astype(np.float32) # Read array
    array[array == src.nodata] = np.nan # Assign np.nan value to all cells that contain a nodata value
    
    affine = src.transform # Affine transform
    profile = src.profile # Metadata
    nodata = src.nodata # Nodata

In [30]:
print(nodata)

-32767.0


In [19]:
# Function: Scaling using a Robust scaler

def scaling_Robust_raster(array):
    scaledArrays = []
    for i in range(len(array)):
        # Scale and Stack
        arrayReshape = array[i].reshape(-1, 1) # Reshape array to 1D-array
        arrayReshape2 = arrayReshape[np.logical_not(np.isnan(arrayReshape))].reshape(-1, 1) # Exclude np.nan during scaling and reshape to 1D-array
        
        scaler = RobustScaler(with_centering=True, with_scaling=True, 
                                           quantile_range=(25.0, 75.0), copy=True).fit(arrayReshape2) # Prepare RobustScaler
        
        # Apply RobustScaler.
        arrayRescaled = scaler.transform(arrayReshape).reshape(array.shape[1], array.shape[2]) # Rescale to shape of original dataset
        scaledArrays.append(arrayRescaled)
    arrayStacked = np.stack(scaledArrays)
    return(arrayStacked)

In [20]:
# Function: Scaling using a min-max scaler

def scaling_MinMax_raster(array):
    scaledArrays = []
    for i in range(len(array)):
        # Scale and Stack
        arrayReshape = array[i].reshape(-1, 1) # Reshape array to 1D-array
        arrayReshape2 = arrayReshape[np.logical_not(np.isnan(arrayReshape))].reshape(-1, 1) # Exclude np.nan during scaling and reshape to 1D-array
        
        scaler = MinMaxScaler(feature_range=(0, 1)).fit(arrayReshape2) # Prepare StandardScaler
        
        # Apply MinMaxScaler.
        arrayRescaled = scaler.transform(arrayReshape).reshape(array.shape[1], array.shape[2]) # Rescale to shape of original dataset
        scaledArrays.append(arrayRescaled)
    arrayStacked = np.stack(scaledArrays)
    return(arrayStacked)

In [27]:
# Run Scaling fucntion
arrayScaledRobust = scaling_Robust_raster(array)
arrayScaledMinMax = scaling_MinMax_raster(arrayScaledRobust)

In [28]:
print(arrayScaledMinMax[0])
print(profile)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -32767.0, 'width': 6301, 'height': 5450, 'count': 10, 'crs': CRS.from_epsg(26911), 'transform': Affine(0.049986166211712445, 0.0, 439206.0313357129,
       0.0, -0.04998678552294406, 6323096.616902698), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}


In [29]:
# Write scaled raster to file

profile['count'] = len(arrayScaledMinMax) # Manipulate dict
profile['nodata'] = np.nan
profile['dtype'] = np.float32

with rasterio.open(rasPathOut, 'w', **profile) as src:
    src.write(arrayScaledMinMax)